In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf gromacs_and_plumed_pytorch
!unzip /content/drive/MyDrive/gromacs_and_plumed_pytorch-20240530T140144Z-001.zip -d .

In [ ]:
!rm -rf plumed_pytorch_tutorial
!git clone https://github.com/dhimanray/plumed_pytorch_tutorial.git

## **Simulation of Toy Model: Mueller Brown Potential**

In [ ]:

#Create the folder with the necessary input files
import os
path = '/content/plumed_pytorch_tutorial/mueller/'
#os.system('rm -r %s'%path)
#os.system('mkdir %s'%path)
os.system('cd %s'%path)

#Copy the MD potential file
#os.system('ln -s /content/drive/MyDrive/input_md-potential.dat %s/'%path)

#Copy the pytorch file
#os.system('ln -s /content/drive/MyDrive/mueller_2state_model.pt %s/'%path)

## PLUMED INPUT FILE FOR ENHANCED SAMPLING (MODIFY AS NEEDED) ##
plumed_input = """
UNITS NATURAL

p: POSITION ATOM=1
mueller: CUSTOM ARG=p.x,p.y PERIODIC=NO ...
  FUNC=0.15*(-200*exp(-(x-1)^2-10*y^2)-100*exp(-x^2-10*(y-0.5)^2)-170*exp(-6.5*(0.5+x)^2+11*(x+0.5)*(y-1.5)-6.5*(y-1.5)^2)+15*exp(0.7*(1+x)^2+0.6*(x+1)*(y-1)+0.7*(y-1)^2)+146.7)
...
potential: BIASVALUE ARG=mueller
lwall: LOWER_WALLS ARG=p.x KAPPA=1000 AT=-1.3
uwall: UPPER_WALLS ARG=p.x KAPPA=1000 AT=+1.0
lwally: LOWER_WALLS ARG=p.y KAPPA=1000 AT=-0.2
uwally: UPPER_WALLS ARG=p.y KAPPA=1000 AT=+2.0

deepTDA_2D: PYTORCH_MODEL FILE=mueller_2state_model.pt ARG=p.x,p.y

opes: OPES_METAD ...
  ARG=p.y
  PACE=500
  BARRIER=30
...


PRINT FMT=%g STRIDE=50 FILE=COLVAR ARG=p.x,p.y,deepTDA_2D.node-0,mueller,opes.bias


ENDPLUMED"""

with open('%s/plumed.dat'%path, 'w') as file:
  file.write(plumed_input)

##Simulation Input file
input_md = """
nstep             2000000
tstep             0.005
temperature       1.0
friction          10.0
random_seed       567
plumed_input      plumed.dat
dimension         2
replicas          1
basis_functions_1 BF_POWERS ORDER=1 MINIMUM=-4.0 MAXIMUM=+3.0
basis_functions_2 BF_POWERS ORDER=1 MINIMUM=-1.0 MAXIMUM=+2.5
input_coeffs       input_md-potential.dat
initial_position   -0.693111,1.40842
output_coeffs           /dev/null
output_potential        /dev/null
output_potential_grid   10
output_histogram        /dev/null
"""
with open('%s/input_md.dat'%path, 'w') as file:
  file.write(input_md)



In [ ]:
%%bash
##------- source plumed pytorch interface -----------------------
rm -r plumed2-master
ln -s gromacs_and_plumed_pytorch/plumed2-master .
chmod +x plumed2-master/*
chmod +x plumed2-master/*/*
chmod +x plumed2-master/src/lib/plumed
source plumed2-master/libtorch/sourceme.sh
source plumed2-master/sourceme.sh
##---------------------------------------------------------------

##copy toy model files
cd /content/plumed_pytorch_tutorial/mueller/

cat input_md-potential.dat

plumed ves_md_linearexpansion input_md.dat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

N = 200 #number of points for plotting/interpolation

m = 40   #number of contours

x, y, z = np.genfromtxt(r'plumed_pytorch_tutorial/mueller/2d_fes', unpack=True)

for i in range(len(z)):
    if z[i] >= 20.0:
        z[i] = 20
xi = np.linspace(x.min(), x.max(), N)
yi = np.linspace(y.min(), y.max(), N)
zi = scipy.interpolate.griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

X, Y = np.meshgrid(xi, yi)
plt.contour(X,Y,zi,colors='black', linewidths=1)

l = np.loadtxt('plumed_pytorch_tutorial/mueller/COLVAR')

plt.scatter(l[:,1],l[:,2],c=l[:,4])

In [ ]:
l = np.loadtxt('plumed_pytorch_tutorial/mueller/COLVAR')

plt.scatter(l[:4000,0],l[:4000,2],c=l[:4000,3])

In [ ]:
#Calculate 1D Free energy profile along x
!cd plumed_pytorch_tutorial/mueller && python FES_from_Reweighting.py --colvar COLVAR --cv p.x --kt 1.0 --sigma 0.05 -o fes_x_test

In [ ]:
fes_x_test = np.loadtxt('plumed_pytorch_tutorial/mueller/fes_x_test')
fes_x_ref = np.loadtxt('plumed_pytorch_tutorial/mueller/fes_x')

plt.plot(fes_x_test[:,0],fes_x_test[:,1])
plt.plot(fes_x_ref[:,0],fes_x_ref[:,1])

In [ ]:
#Calculate 1D Free energy profile along y axis
!cd plumed_pytorch_tutorial/mueller && python FES_from_Reweighting.py --colvar COLVAR --cv p.y --kt 1.0 --sigma 0.05 -o fes_y_test

In [ ]:
fes_y_test = np.loadtxt('plumed_pytorch_tutorial/mueller/fes_y_test')
fes_y_ref = np.loadtxt('plumed_pytorch_tutorial/mueller/fes_y')

plt.plot(fes_y_test[:,0],fes_y_test[:,1])
plt.plot(fes_y_ref[:,0],fes_y_ref[:,1])

## **Beginning of the Machine Learning CV discovery**
We will do the Deep-TICA CV here

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#Install pytorch and libtorch
!conda install pytorch==2.2.2 torchvision==0.17.2 -c pytorch


In [ ]:
import torch
print(torch.__version__)

In [ ]:
!pip install mlcolvar

In [ ]:

import pandas as pd
from mlcolvar.utils.io import load_dataframe
from mlcolvar.utils.timelagged import create_timelagged_dataset
from mlcolvar.data import DictModule

from google.colab import drive


# load colvar files containing weights of the OPES simulations as well as the driver with the input features
df = load_dataframe('plumed_pytorch_tutorial/mueller/COLVAR',
                      start=0,stop=10000)


df

In [ ]:
# Select input features
X = df.filter(regex='p.x|p.y').values
n_input = X.shape[1]

print(X.shape)

### Create time-lagged dataset

**Compute weights for time rescaling**

Here we extract the time $t$, the energy $E$ (needed for the multicanonical reweight [1]) and the bias $V$ from the COLVAR file. We then calculate the weights as:

\begin{cases}
    w = e^{\beta\ V + (\beta_0-\beta)\ E} & \text{if multicanonical}\\
    w = e^{\beta\ V}              & \text{otherwise}
\end{cases}

NB: note that if simulation temperature $\beta_0$ is equal to the reweighting one $\beta$ the multicanonical reweight coincides with the standard umbrella sampling-like case.

Once we have computed the weights, we rescale the time at step $k$ by using the instantaneus acceleration:

$$ dt'_k = w_k\ dt $$

and then compute the cumulative rescaled time:

$$ t'_k = \sum_{i=0} ^k dt'_i $$



[1] Invernizzi, Piaggi, and Parrinello. "Unified approach to enhanced sampling." _Physical Review X_ 10.4 (2020): 041034.


---

| Parameter | Type | Description |
| :- | :- | :- |
| multicanonical | bool | flag to determine if using a standard reweight (false) or a multicanonical one (true) |
| temp | float | reweighting temperature |
| temp0 | float | simulation temperature (only needed if multicanonical == True) |




In [ ]:
#------------- PARAMETERS -------------
multicanonical    = False
temp              = 1.0
temp0             = 1.0
#--------------------------------------

# Calculate inverse temperature
kb=1.0
beta=1./(kb*temp)
beta0=1./(kb*temp0)

# Extract cvs from df

t = df['time'].values # save time
#ene = df['ene'].values.astype(np.float64) # store energy as long double
bias = df.filter(regex='.bias').values.sum(axis=1) # Load *.bias columns and sum them

# Compute log-weights for time reweighting
logweights = beta*bias
#logweights = 0.0*bias

if multicanonical:
    ene -= np.mean(ene) #first shift energy by its mean value
    logweights += (beta0-beta)*ene

In order to train the Deep-TICA CVs we will need to compute the time-lagged covariance matrices in the rescaled time $t'$. The standard way is to look for configurations which are distant a lag-time $\tau$ in the time series. However, in the rescaled time the time-series is _exponentially_ unevenly spaced. Hence, a naive search will lead to severe numerical issue. To address this, we use the algorithm proposed in [2]. In a nutshell, this method assume that the observable $O(t'_k)$ have the same value from scaled time $t'_k$ to $t'_{k+1}$. This leads to weighting each pair of configurations based both on the rescaled time around $t'_k$ and $t'_k+\tau$ (see supp. information of [2] for details). All of this is done under the hood in the function `find_time_lagged_configurations`. To generate the training set we use the function `create_time_lagged_dataset` which searches for the pairs of configurations and the corresponding weight.

[2] Yang and Parrinello. "Refining collective coordinates and improving free energy representation in variational enhanced sampling." _Journal of chemical theory and computation_ 14.6 (2018): 2889-2894.

---

| Parameter | Type | Description |
| :- | :- | :- |
| lag_time | float | lag_time for the calculation of the covariance matrices [in rescaled time] |
| n_train | int | number of training configurations |
| n_valid | int | number of validation configurations |


In [ ]:
#------------- PARAMETERS -------------
lag_time = 1.
#--------------------------------------

# create dataset
dataset = create_timelagged_dataset(X,t,lag_time=lag_time,logweights=logweights,progress_bar=True)

# create datamodule (split train valid)
datamodule = DictModule(dataset,lengths=[0.8,0.2],random_split=False,shuffle=False)

datamodule

### Define model

In [ ]:
from mlcolvar.cvs import DeepTICA

n_components = 2
nn_layers = [n_input, 10, 10, n_components]
options= {'nn': {'activation': 'shifted_softplus'}}

model = DeepTICA(nn_layers, options=options)
model

### Define Trainer & Fit

In [ ]:
import lightning
import torch
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from mlcolvar.utils.trainer import MetricsCallback

# define callbacks
metrics = MetricsCallback()
early_stopping = EarlyStopping(monitor="valid_loss", min_delta=1e-3, patience=100)

# define trainer
trainer = lightning.Trainer(callbacks=[metrics, early_stopping],
                     max_epochs=None, logger=None, enable_checkpointing=False)

# fit
torch.set_float32_matmul_precision('medium')
trainer.fit( model, datamodule )

We can monitor how the individual eigenvalues are optimized during training.

In [ ]:
from mlcolvar.utils.plot import plot_metrics

ax = plot_metrics(metrics.metrics,
                  keys=[x for x in  metrics.metrics.keys() if 'valid_eigval' in x],#['train_loss_epoch','valid_loss'],
                  #linestyles=['-.','-'], colors=['fessa1','fessa5'],
                  yscale='linear')

### Normalize output

For convenience, we standardize the CVs to be between -1 and 1.

In [ ]:
from mlcolvar.core.transform import Normalization
from mlcolvar.core.transform.utils import Statistics

#X = dataset[:]['data']
with torch.no_grad():
    model.postprocessing = None # reset
    s = model(torch.Tensor(X))

norm =  Normalization(n_components, mode='min_max', stats = Statistics(s) )
model.postprocessing = norm

### Compute FES

We can plot the free energy profile along the selected CVs. First we compute it in the 2D space, from which we learn that the first TICA refers to the transition between two long-lived metastable states, while the second the faster transitions within the left basin.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mlcolvar.utils.fes import compute_fes

fig,ax = plt.subplots(1,1,figsize=(6,5),dpi=100)

# compute cvs
with torch.no_grad():
    s = model(torch.Tensor(X)).numpy()

w = np.exp(logweights)
fes,grid,bounds,error = compute_fes(s,
                                    weights=w,
                                    blocks=1,
                                    bandwidth=0.01, scale_by='range',
                                    plot=True, plot_max_fes=50, ax = ax, eps=1e-10)

ax.set_xlabel('Deep-TICA 1')
ax.set_ylabel('Deep-TICA 2')

We can also plot the free energy profiles along each component. From the 2D FES above we understand that for the second CV we need to compute the FES only for the points in which the first CV is < 0, in order to describe the barrier between the faster states.

In [ ]:
from mlcolvar.utils.fes import compute_fes

fig,axs = plt.subplots(1,n_components,figsize=(6*n_components,5),dpi=100)

for i in range(n_components):
    w = np.exp(logweights)

    # restrict the second CV to the points in which the first is < 0
    fes,grid,bounds,error = compute_fes(s[:,i] if i == 0 else s[s[:,0] < 0, i ],
                                        weights=w if i == 0 else w[s[:,0] < 0 ],
                                        blocks=2,
                                        bandwidth=0.02,scale_by='range',
                                        plot=True, plot_max_fes=100, ax = axs[i])
    axs[i].set_xlabel('Deep-TICA '+str(i+1))

Note that the huge uncertainty on the free energy barrier is due to the lack of points between the states in the multithermal simulation.

### Plot CVs isolines in $\phi-\psi$ space

To understand to what states the CVs refer to, we can visualize them in the Ramachandran plot ($\phi$ and $\psi$).

In [ ]:
# Hexbin plot in physical space
fig,axs = plt.subplots(1,n_components,figsize=(6*n_components,5),dpi=100)

x = df['p.x'].values
y = df['p.y'].values

# compute cvs
with torch.no_grad():
    s = model(torch.Tensor(X)).numpy()

for i,ax in enumerate(axs):
    pp = ax.hexbin(x,y,C=s[:,i],gridsize=150,cmap='fessa')
    cbar = plt.colorbar(pp,ax=ax)
    ax.set_title('Deep-TICA '+str(i+1))
    ax.set_xlabel(r'p.x')
    ax.set_ylabel(r'p.y')
    cbar.set_label('Deep-TICA '+str(i+1))

plt.tight_layout()
plt.show()

In [ ]:
## Save the model
x = torch.rand(2, dtype=torch.float32, requires_grad=True).unsqueeze(0)
traced_cv = torch.jit.trace ( model, example_inputs=x)
filename='plumed_pytorch_tutorial/mueller_deeptica.pt'
traced_cv.save(filename)

## **Simulation using the Deep-TICA CV**

In [ ]:

#Create the folder with the necessary input files
import os
path = '/content/plumed_pytorch_tutorial/mueller_deeptica'
#os.system('cd %s'%path)
#os.system('echo pwd')
os.system('rm -r %s'%path)
os.system('mkdir %s'%path)
#os.system('cd mueller_deeptica')

#Copy the MD potential file
os.system('ln -s /content/plumed_pytorch_tutorial/mueller/input_md-potential.dat %s/'%path)

#Copy the pytorch file
os.system('ln -s /content/plumed_pytorch_tutorial/mueller_deeptica.pt %s/'%path)

## PLUMED INPUT FILE FOR ENHANCED SAMPLING (MODIFY AS NEEDED) ##
plumed_input = """
UNITS NATURAL

p: POSITION ATOM=1
mueller: CUSTOM ARG=p.x,p.y PERIODIC=NO ...
  FUNC=0.15*(-200*exp(-(x-1)^2-10*y^2)-100*exp(-x^2-10*(y-0.5)^2)-170*exp(-6.5*(0.5+x)^2+11*(x+0.5)*(y-1.5)-6.5*(y-1.5)^2)+15*exp(0.7*(1+x)^2+0.6*(x+1)*(y-1)+0.7*(y-1)^2)+146.7)
...
potential: BIASVALUE ARG=mueller
lwall: LOWER_WALLS ARG=p.x KAPPA=1000 AT=-1.3
uwall: UPPER_WALLS ARG=p.x KAPPA=1000 AT=+1.0
lwally: LOWER_WALLS ARG=p.y KAPPA=1000 AT=-0.2
uwally: UPPER_WALLS ARG=p.y KAPPA=1000 AT=+2.0

deepTICA: PYTORCH_MODEL FILE=mueller_deeptica.pt ARG=p.x,p.y

opes: OPES_METAD ...
  ARG=deepTICA.node-0
  PACE=500
  BARRIER=30
...


PRINT FMT=%g STRIDE=50 FILE=COLVAR ARG=p.x,p.y,deepTICA.node-0,mueller,opes.bias


ENDPLUMED"""

with open('%s/plumed.dat'%path, 'w') as file:
  file.write(plumed_input)

##Simulation Input file
input_md = """
nstep             200000
tstep             0.005
temperature       1.0
friction          10.0
random_seed       567
plumed_input      plumed.dat
dimension         2
replicas          1
basis_functions_1 BF_POWERS ORDER=1 MINIMUM=-4.0 MAXIMUM=+3.0
basis_functions_2 BF_POWERS ORDER=1 MINIMUM=-1.0 MAXIMUM=+2.5
input_coeffs       input_md-potential.dat
initial_position   -0.693111,1.40842
output_coeffs           /dev/null
output_potential        /dev/null
output_potential_grid   10
output_histogram        /dev/null
"""
with open('%s/input_md.dat'%path, 'w') as file:
  file.write(input_md)

In [ ]:
%%bash
##------- source plumed pytorch interface -----------------------
rm -r plumed2-master
ln -s gromacs_and_plumed_pytorch/plumed2-master .
chmod +x plumed2-master/*
chmod +x plumed2-master/*/*
chmod +x plumed2-master/src/lib/plumed
source plumed2-master/libtorch/sourceme.sh
source plumed2-master/sourceme.sh
##---------------------------------------------------------------

##copy toy model files
cd /content/plumed_pytorch_tutorial/mueller_deeptica/
plumed ves_md_linearexpansion input_md.dat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

N = 200 #number of points for plotting/interpolation

m = 40   #number of contours

x, y, z = np.genfromtxt(r'plumed_pytorch_tutorial/mueller/2d_fes', unpack=True)

for i in range(len(z)):
    if z[i] >= 20.0:
        z[i] = 20
xi = np.linspace(x.min(), x.max(), N)
yi = np.linspace(y.min(), y.max(), N)
zi = scipy.interpolate.griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

X, Y = np.meshgrid(xi, yi)
plt.contour(X,Y,zi,colors='black', linewidths=1)

l = np.loadtxt('plumed_pytorch_tutorial/mueller_deeptica/COLVAR')

plt.scatter(l[:,1],l[:,2],c=l[:,4])

In [ ]:
l = np.loadtxt('plumed_pytorch_tutorial/mueller_deeptica/COLVAR')

plt.scatter(l[:,0],l[:,2],c=l[:,3])

In [ ]:
#Calculate 1D Free energy profile along x
!cd plumed_pytorch_tutorial/mueller_deeptica && python ../mueller/FES_from_Reweighting.py --colvar COLVAR --cv p.x --kt 1.0 --sigma 0.05 -o fes_x_test

In [ ]:
fes_x_test = np.loadtxt('plumed_pytorch_tutorial/mueller_deeptica/fes_x_test')
fes_x_ref = np.loadtxt('plumed_pytorch_tutorial/mueller/fes_x')

plt.plot(fes_x_test[:,0],fes_x_test[:,1])
plt.plot(fes_x_ref[:,0],fes_x_ref[:,1])

In [ ]:
#Calculate 1D Free energy profile along y axis
!cd plumed_pytorch_tutorial/mueller_deeptica && python ../mueller/FES_from_Reweighting.py --colvar COLVAR --cv p.y --kt 1.0 --sigma 0.05 -o fes_y_test

In [ ]:
fes_y_test = np.loadtxt('plumed_pytorch_tutorial/mueller_deeptica/fes_y_test')
fes_y_ref = np.loadtxt('plumed_pytorch_tutorial/mueller/fes_y')

plt.plot(fes_y_test[:,0],fes_y_test[:,1])
plt.plot(fes_y_ref[:,0],fes_y_ref[:,1])